# ACESSIBILIDADE DA REDE DE VIÇOSA (SEM ANEL)

### IMPORTANDO O PACOTE psycopg2 QUE CONECTA O PYTHON COM O POSTGRE-SQL E O PACOTE pandas PARA ORGANIZAR AS TABELAS DE ACESSIBILIDADE

In [1]:
import psycopg2 as pg
import pandas as pd

### CONECTANDO AO BANCO DE DADOS

In [2]:
con = pg.connect(host='localhost', 
                database='dissertacao_artigo1',
                user='postgres', 
                password='admin')

cur = con.cursor() #CRIANDO UMA INSTÂNCIA PARA EXECUTAR COMANDOS EM SQL

# OBS: O servidor hospedado na máquina local será conectado no banco de dados nomeado rede_exemplo, que possui usuário postgres e senha admin.

### VENDO QUANTOS GRAFOS A MALHA POSSUI

In [3]:
tabela_grafos = 'rede_vicosa' #TABELA COM A REDE
sql = f'select max(id) from {tabela_grafos}' #COMANDO EM SQL A SER EXECUTADO
cur.execute(sql) #EXECUTANDO O COMANDO CRIADO
dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
id_max = e = dados_consultados[0][0] #ID MÁXIMO DA MALHA

### VENDO QUANTOS VÉRTICES A MALHA POSSUI 

In [4]:
tabela_vertices = 'rede_vicosa_vertices_pgr' #TABELA COM A REDE
sql = f'select max(id) from {tabela_vertices}' #COMANDO EM SQL A SER EXECUTADO
cur.execute(sql) #EXECUTANDO O COMANDO CRIADO
dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
v = dados_consultados[0][0] #ID MÁXIMO DA MALHA

### CONECTIVIDADE DA MALHA

In [5]:
#INDICE ALFA: NÚMERO DE CLICOS DA REDE 'u'
#FORMULA:    u = e-v+1
#            u_max = 2v-5
#            alfa = u/u_max ou (e-v+1) / (2v-5)
#            sendo e: numero de linhas; v: numero de vértices

#INDICE BETA: SIMPLES RELAÇÃO ENTRE NUMERO DE LINHAS E VÉRTICES
#FORMULA:    beta = e / v

#INDICE GAMA: RELAÇÃO NUMERO DE LINHAS OBSERVADAS E O NUMERO MÁXIMO DE LINHAS
#FORMULA:    gama = e / (3(v-2))

alfa = (e-v+1)/(2*v-5)
beta = e/v
gama = e/(3*(v-2))

print(f'No cálculo de conectividade da rede, obtemos os seguintes resultados:\nÍndice Alfa: {alfa:.3f}\nÍndice Beta: {beta:.3f}\nÍndice Gama: {gama:.3f}')

No cálculo de conectividade da rede, obtemos os seguintes resultados:
Índice Alfa: 0.200
Índice Beta: 1.376
Índice Gama: 0.470


### NOME DAS TABELAS 

In [6]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos_acess = 'rede_vicosa_mp' #NOME DE TABELA DA REDE
abr_grafos = 'rvmp' #ABREVIAÇÃO PARA A TABELA DA REDE
lista_custos = []

### LIGAÇÕES ENTRE OS GRAFOS

In [7]:
for id_i in range(1001, id_max + 1001): #ITERAÇÃO QUE IRÁ PERCORRER TODOS OS GRAFOS COMO VÉRTICE INICIAL

    lista_custos_i = [] #CRIANDO UMA LISTA VAZIA PARA RECEBER OS CUSTOS DO GRAFO i
    
    for id_j in range(1001, id_max + 1001): #ITERAÇÃO QUE IRÁ PERCORRER TODOS OS GRAFOS COMO VÉRTICE FINAL
        
        if id_i == id_j: #SE A ITERAÇÃO CALCULAR A DISTANCIA DE UM GRAFO PARA ELE MESMO, PULA A ITERAÇÃO E ADICIONA CUSTO ZERO NA LISTA
            lista_custos_i.append(0)
            continue
        
        else:

            sql = f"SELECT sum(djk.cost)/2 as tot_cost FROM pgr_dijkstra('SELECT id, source, target, cost, reverse_cost from {tabela_grafos_acess}', {id_i}, {id_j}, true) as djk JOIN {tabela_grafos_acess} {abr_grafos} ON djk.edge = {abr_grafos}.id;" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O VÉRTICE INICIAL E FINAL DO GRAFO DO OBJETIVO FINAL.

            cur.execute(sql) #EXECUTANDO O COMANDO

            dados_consultados = cur.fetchall() #RETORNANDO OS DADOS            
                      
            cost = int(dados_consultados[0][0]) #CUSTO ENTRE i E j
            
            #ADICIONANDO OS DADOS EM UMA LISTA:
            lista_custos_i.append(cost) #ADICIONANDO O CUSTO DE ATRAVESSAR DO GRAFO i ATÉ O GRAFO j EM UMA LISTA PROVISÓRIA

    lista_custos.append(lista_custos_i) #ADICIONANDO A TABELA PROVISIÓRIA ACIMA EM UMA LISTA QUE TERÁ TODAS INFORMAÇÕES


### TRANSFORMANDO A LISTA EM TABELA COM O PANDAS 

In [8]:
#CRIANDO UMA MATRIZ A PARTIR DA LISTA ACIMA:
matrizCustos = pd.DataFrame(lista_custos)

#SUBSTITUINDO O CABEÇALHO E LINHAS QUE ESTÁ INDO DE 0 ATÉ 7 PARA OS NOMES DOS GRAFOS QUE VAI DE A ATÉ H:
matrizCustos_ren = matrizCustos #CRIANDO UMA COPIA DA MATRIZ ANTIGA, PARA PRESERVAR A ESTRUTURA ORIGINAL

for i in range(0, len(matrizCustos)+1): #ITERAÇÃO PARA SUBSTITUIR O NOME DO CABEÇALHO E DAS LINHAS
    matrizCustos_ren = matrizCustos_ren.rename(columns={i: f'{i+1}'}, index = {i: f'{i+1}'})

In [9]:
matrizCustos

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
0,0,1,1,13,13,12,13,13,12,11,...,9,5,6,7,7,8,8,8,8,9
1,1,0,2,13,13,12,13,12,12,11,...,10,6,7,8,8,9,9,9,9,8
2,2,1,0,12,12,11,12,13,11,10,...,8,4,5,6,6,7,7,7,7,8
3,13,12,11,0,1,1,2,2,2,2,...,7,11,10,9,9,9,8,8,9,8
4,13,12,11,1,0,1,1,1,2,2,...,7,11,10,9,9,9,8,8,9,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,10,9,8,9,9,8,9,10,8,7,...,2,3,2,1,2,0,1,2,3,7
113,10,9,8,8,8,7,8,9,7,6,...,1,3,2,1,2,1,0,1,2,6
114,10,9,8,8,8,7,8,9,7,6,...,1,3,2,2,1,2,1,0,1,6
115,10,9,8,9,9,8,9,10,8,7,...,2,3,2,2,1,3,2,1,0,7


### Matriz dos custos de grafo a grafo 

In [10]:
matrizCustos_ren

,1,2,3,4,5,6,7,8,9,10,...,108,109,110,111,112,113,114,115,116,117
1,0,1,1,13,13,12,13,13,12,11,...,9,5,6,7,7,8,8,8,8,9
2,1,0,2,13,13,12,13,12,12,11,...,10,6,7,8,8,9,9,9,9,8
3,2,1,0,12,12,11,12,13,11,10,...,8,4,5,6,6,7,7,7,7,8
4,13,12,11,0,1,1,2,2,2,2,...,7,11,10,9,9,9,8,8,9,8
5,13,12,11,1,0,1,1,1,2,2,...,7,11,10,9,9,9,8,8,9,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,10,9,8,9,9,8,9,10,8,7,...,2,3,2,1,2,0,1,2,3,7
114,10,9,8,8,8,7,8,9,7,6,...,1,3,2,1,2,1,0,1,2,6
115,10,9,8,8,8,7,8,9,7,6,...,1,3,2,2,1,2,1,0,1,6
116,10,9,8,9,9,8,9,10,8,7,...,2,3,2,2,1,3,2,1,0,7


### Acessibilidade pelo Método 1:

In [11]:
matrizAcess_conectiv = matrizCustos_ren.filter(items=list(map(lambda x: f'{x}', range(1, id_max + 1))))\
                                        .where(matrizCustos_ren.values == 1) #SELECIONANDO APENAS OS GRAFOS QUE POSSUEM LIGAÇÕES DIRETAS

matrizAcess_conectiv = matrizAcess_conectiv.apply(lambda x: x.replace(float('NaN'), 0)) #ATRIBUINDO VALOR ZERO PARA OS GRAFOS SEM LIGAÇÕES DIRETAS

In [12]:
matrizAcess_conectiv #MATRIZ DE CONECTIVADE

,1,2,3,4,5,6,7,8,9,10,...,108,109,110,111,112,113,114,115,116,117
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [13]:
Acess_1 = matrizAcess_conectiv.sum(axis=1) #SOMA DAS LINHAS DA MATRIZ DE CONECTIVIDADE
Acess_1 = pd.DataFrame(Acess_1).rename(columns = {0: 'SOMATÓRIO'}) #CRIANDO UM DATA FRAME PARA RECEBER OS DADOS

In [14]:
Acess_1

,SOMATÓRIO
1,2.0
2,2.0
3,3.0
4,2.0
5,4.0
...,...
113,2.0
114,4.0
115,4.0
116,2.0


### Acessibilidade pelo Método 2: 

In [15]:
Acess_2 = matrizCustos_ren.max() #VALORES MÁXIMOS PARA CADA GRAFO ATÉ O GRAFO MAIS DISTANTE NA REDE (NÚMERO ASSOCIADO)
Acess_2 = pd.DataFrame(Acess_2).rename(columns = {0: 'SOMATÓRIO'}) #NÚMERO ASSOCIADO

In [16]:
Acess_2

,SOMATÓRIO
1,14
2,14
3,14
4,17
5,17
...,...
113,20
114,19
115,19
116,20


### Acessibilidade pelo Método 3: 

In [17]:
# A ORDEM DE LIGAÇÃO MÁXIMA PODE SER OBTIDADE DA TABELA DO CÁLCULO DO NÚMERO ASSOCIADO, SENDO O VALOR MÁXIMO OBTIDO NO CALCULO DE ACESSIBILIDADE DO MÉTODO 2
lig_max = int(Acess_2.max())

lista_acess_3 = [] #CRIANDO UMA LISTA QUE RECEBERÁ O SOMATÓRIO DA ACESSIBILIDADE PARA CADA ORDEM

for i in range(1, lig_max + 1): #ITERAÇÃO QUE IRÁ PERCORRER DA ORDEM 1 ATÉ A ORDEM MÁXIMA
    
    matrizAcess_ordem_i = matrizCustos_ren.filter(items=list(map(lambda x: f'{x}', range(1, id_max + 1))))\
                                            .where(matrizCustos_ren.values == i) #SELECIONANDO APENAS OS GRAFOS QUE POSSUEM LIGAÇÕES DE ORDEM i
    
    matrizAcess_ordem_i = matrizAcess_ordem_i.apply(lambda x: x.replace(float('NaN'), 0)) #DEFININDO OS VALORES 'NaN' IGUAL A ZERO
    
    sumMatrizAcess_ordem_i = matrizAcess_ordem_i.sum(axis=1) #SOMANDO AS LINHAS
    
    lista_acess_3.append(list(sumMatrizAcess_ordem_i)) #ADICIONANDO O SOMATÓRIO A LISTA DE ACESSIBILIDADE 3

Acess_3 = pd.DataFrame(pd.DataFrame(lista_acess_3).sum()) #DEFININDO ACESSIBILIDADE PELO MÉTODO TRÊS COMO O SOMATÓRIO DE TODAS ACESSIBILIDADES DE ORDEM N

#RENOMEANDO AS LINHAS E COLUNAS DESSA MATRIZ:


Acess_3 = Acess_3.rename(columns = {0: 'SOMATÓRIO'}) #DEFININDO O NOME DA COLUNA

for i in range(0, len(Acess_3)): #ITERAÇÃO PARA SUBSTITUIR DAS LINHAS
    Acess_3 = Acess_3.rename(index = {i: f'{i+1}'})
    

## RESULTADOS: 

### Acessibilidade Método 1: 

In [18]:
Acess_1

,SOMATÓRIO
1,2.0
2,2.0
3,3.0
4,2.0
5,4.0
...,...
113,2.0
114,4.0
115,4.0
116,2.0


### Acessibilidade Método 2: 

In [19]:
Acess_2

,SOMATÓRIO
1,14
2,14
3,14
4,17
5,17
...,...
113,20
114,19
115,19
116,20


### Acessibilidade Método 3: 

In [20]:
Acess_3

,SOMATÓRIO
1,960.0
2,921.0
3,915.0
4,1096.0
5,1090.0
...,...
113,1160.0
114,1064.0
115,1064.0
116,1160.0


## Adicionando as Acessibilidades na tabela rede_vicosa: 

In [21]:
for id_i in range (1, id_max + 1):

    sql = f"update {tabela_grafos} set acess_1 = '{Acess_1.values[id_i-1][0]}', acess_2 = '{Acess_2.values[id_i-1][0]}', acess_3 = '{Acess_3.values[id_i-1][0]}' where id = {id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA 'REDE_VICOSA' AS ACESSIBILIDADES DAS VIAS.

    cur.execute(sql) #EXECUTANDO O COMANDO

    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
   
    print(f'Grafo id {id_i}: Acessibilidade 1: {Acess_1.values[id_i-1][0]}; Acessibilidade 2: {Acess_2.values[id_i-1][0]}; Acessibilidade 3: {Acess_3.values[id_i-1][0]};')
    
cur.close() #ENCERRANDO A INSTÂNCIA CRIADA PARA A EXECUÇÃO DO COMANDO

con.close() #ENCERRANDO A CONEXÃO COM BANCO DE DADOS

Grafo id 1: Acessibilidade 1: 2.0; Acessibilidade 2: 14; Acessibilidade 3: 960.0;
Grafo id 2: Acessibilidade 1: 2.0; Acessibilidade 2: 14; Acessibilidade 3: 921.0;
Grafo id 3: Acessibilidade 1: 3.0; Acessibilidade 2: 14; Acessibilidade 3: 915.0;
Grafo id 4: Acessibilidade 1: 2.0; Acessibilidade 2: 17; Acessibilidade 3: 1096.0;
Grafo id 5: Acessibilidade 1: 4.0; Acessibilidade 2: 17; Acessibilidade 3: 1090.0;
Grafo id 6: Acessibilidade 1: 4.0; Acessibilidade 2: 16; Acessibilidade 3: 986.0;
Grafo id 7: Acessibilidade 1: 4.0; Acessibilidade 2: 17; Acessibilidade 3: 1090.0;
Grafo id 8: Acessibilidade 1: 4.0; Acessibilidade 2: 16; Acessibilidade 3: 1080.0;
Grafo id 9: Acessibilidade 1: 4.0; Acessibilidade 2: 16; Acessibilidade 3: 984.0;
Grafo id 10: Acessibilidade 1: 4.0; Acessibilidade 2: 15; Acessibilidade 3: 881.0;
Grafo id 11: Acessibilidade 1: 2.0; Acessibilidade 2: 19; Acessibilidade 3: 1407.0;
Grafo id 12: Acessibilidade 1: 4.0; Acessibilidade 2: 19; Acessibilidade 3: 1296.0;
Grafo i